#### Prepare train data of RNet


In [1]:
import argparse


import cv2
import numpy as np
import sys
import os
sys.path.append(os.getcwd())
from mtcnn.core.detect import MtcnnDetector,create_mtcnn_net
from mtcnn.core.imagedb import ImageDB
from mtcnn.core.image_reader import TestImageLoader
import time

from six.moves import cPickle
from mtcnn.core.utils import convert_to_square,IoU
import mtcnn.config as config
import mtcnn.core.vision as vision

prefix_path = ''

traindata_store = '../image/train'

pnet_model_file = './model_store/pnet_epoch.pt'

annotation_file = './anno_store/anno_train_test.txt'

use_cuda = False


ModuleNotFoundError: No module named 'mtcnn'